In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb

import sys, os, gc, types
import time
from subprocess import check_output

import sklearn
from sklearn.model_selection import train_test_split

In [3]:
root_paths = [
    "/data/kaggle-wikipedia/data2/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data2/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data2/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

date_all_df.pkl
date_df.csv
key_1.csv
key_1_modified.csv
key_1_modified.pkl
median_49.csv
median_weekday.csv
median_weekday_fix.csv
median_weekend.csv
page_df.csv
page_df.pkl
page_ohe.csv
page_ohe.pkl
sample_submission_1.csv
spider.txt
states.nn14-r1-1.npy
states.nn14-r1.npy
test_date_df.csv
test_prediction.gb3-r2.csv
test_prediction.gb4-r0.csv
test_prediction.gb4-r1.csv
train_1.csv



In [8]:
median_data = []
median_name = [
    '49_fix', 'weekday_fix', 'weekend_fix', 
    'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5', 'dow6',
    '28', 'weekday_28', 'weekend_28'
]
for mname in median_name:
    median_data.append(pd.read_pickle(root + 'median_{}.pkl'.format(mname)))
    
train = pd.read_csv(root + 'train_2.csv')
# train.fillna(0, inplace = True)
# train = train.where(train.notnull(), median_data[0])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data/median_49_fix.pkl'

In [23]:
date_df = pd.read_csv(root + 'date_df.csv')
page_df = pd.read_pickle(root + 'page_ohe.pkl')

In [24]:
# rnn_feature = np.load(root+'states.nn14-r1-1.npy')
# columns = ['rnn_{}'.format(i) for i in range(300)]
# page_df = pd.concat([page_df, pd.DataFrame(rnn_feature, columns=columns)], axis=1)

In [28]:
page_df.head()

,Page,domain_commons.wikimedia.org,domain_de.wikipedia.org,domain_en.wikipedia.org,domain_es.wikipedia.org,domain_fr.wikipedia.org,domain_ja.wikipedia.org,domain_ru.wikipedia.org,domain_www.mediawiki.org,domain_zh.wikipedia.org,access_all-access,access_desktop,access_mobile-web,agent_all-agents,agent_spider,rnn_0,rnn_1,rnn_2,rnn_3,rnn_4,rnn_5,rnn_6,rnn_7,rnn_8,rnn_9,rnn_10,rnn_11,rnn_12,rnn_13,rnn_14,rnn_15,rnn_16,rnn_17,rnn_18,rnn_19,rnn_20,rnn_21,rnn_22,rnn_23,rnn_24,rnn_25,rnn_26,rnn_27,rnn_28,rnn_29,rnn_30,rnn_31,rnn_32,rnn_33,rnn_34,rnn_35,rnn_36,rnn_37,rnn_38,rnn_39,rnn_40,rnn_41,rnn_42,rnn_43,rnn_44,rnn_45,rnn_46,rnn_47,rnn_48,rnn_49,rnn_50,rnn_51,rnn_52,rnn_53,rnn_54,rnn_55,rnn_56,rnn_57,rnn_58,rnn_59,rnn_60,rnn_61,rnn_62,rnn_63,rnn_64,rnn_65,rnn_66,rnn_67,rnn_68,rnn_69,rnn_70,rnn_71,rnn_72,rnn_73,rnn_74,rnn_75,rnn_76,rnn_77,rnn_78,rnn_79,rnn_80,rnn_81,rnn_82,rnn_83,rnn_84,rnn_85,rnn_86,rnn_87,rnn_88,rnn_89,rnn_90,rnn_91,rnn_92,rnn_93,rnn_94,rnn_95,rnn_96,rnn_97,rnn_98,rnn_99,rnn_100,rnn_101,rnn_102,rnn_103,rnn_104,rnn_105,rnn_106,rnn_107,rnn_108,rnn_109,rnn_110,rnn_111,rnn_112,rnn_113,rnn_114,rnn_115,rnn_116,rnn_117,rnn_118,rnn_119,rnn_120,rnn_121,rnn_122,rnn_123,rnn_124,rnn_125,rnn_126,rnn_127,rnn_128,rnn_129,rnn_130,rnn_131,rnn_132,rnn_133,rnn_134,rnn_135,rnn_136,rnn_137,rnn_138,rnn_139,rnn_140,rnn_141,rnn_142,rnn_143,rnn_144,rnn_145,rnn_146,rnn_147,rnn_148,rnn_149,rnn_150,rnn_151,rnn_152,rnn_153,rnn_154,rnn_155,rnn_156,rnn_157,rnn_158,rnn_159,rnn_160,rnn_161,rnn_162,rnn_163,rnn_164,rnn_165,rnn_166,rnn_167,rnn_168,rnn_169,rnn_170,rnn_171,rnn_172,rnn_173,rnn_174,rnn_175,rnn_176,rnn_177,rnn_178,rnn_179,rnn_180,rnn_181,rnn_182,rnn_183,rnn_184,rnn_185,rnn_186,rnn_187,rnn_188,rnn_189,rnn_190,rnn_191,rnn_192,rnn_193,rnn_194,rnn_195,rnn_196,rnn_197,rnn_198,rnn_199,rnn_200,rnn_201,rnn_202,rnn_203,rnn_204,rnn_205,rnn_206,rnn_207,rnn_208,rnn_209,rnn_210,rnn_211,rnn_212,rnn_213,rnn_214,rnn_215,rnn_216,rnn_217,rnn_218,rnn_219,rnn_220,rnn_221,rnn_222,rnn_223,rnn_224,rnn_225,rnn_226,rnn_227,rnn_228,rnn_229,rnn_230,rnn_231,rnn_232,rnn_233,rnn_234,rnn_235,rnn_236,rnn_237,rnn_238,rnn_239,rnn_240,rnn_241,rnn_242,rnn_243,rnn_244,rnn_245,rnn_246,rnn_247,rnn_248,rnn_249,rnn_250,rnn_251,rnn_252,rnn_253,rnn_254,rnn_255,rnn_256,rnn_257,rnn_258,rnn_259,rnn_260,rnn_261,rnn_262,rnn_263,rnn_264,rnn_265,rnn_266,rnn_267,rnn_268,rnn_269,rnn_270,rnn_271,rnn_272,rnn_273,rnn_274,rnn_275,rnn_276,rnn_277,rnn_278,rnn_279,rnn_280,rnn_281,rnn_282,rnn_283,rnn_284,rnn_285,rnn_286,rnn_287,rnn_288,rnn_289,rnn_290,rnn_291,rnn_292,rnn_293,rnn_294,rnn_295,rnn_296,rnn_297,rnn_298,rnn_299
0,2NE1_zh.wikipedia.org_all-access_spider,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0.037819,-0.080940,0.368670,0.015821,-0.014308,-0.024649,0.054147,0.033753,0.003721,-0.072414,0.046213,0.029239,0.079015,-0.085323,-0.004968,-0.005087,0.059767,-0.026677,-0.024340,0.006248,0.016026,0.284250,-0.009716,0.239139,0.061682,-0.096446,-0.088631,0.002439,-0.027581,-0.307261,-0.017020,-0.020336,0.541521,0.040400,-0.001624,9.444691,0.205280,-0.058978,-0.094876,-0.039772,0.002273,-0.013713,-0.259176,0.029907,0.063862,-0.008944,0.036521,-0.008684,0.002812,0.067283,0.024010,-0.025872,-0.024110,0.079990,-0.014442,0.019859,0.001714,-0.010817,0.022679,0.006708,0.037228,0.220925,0.019854,0.063461,-0.028861,-0.015207,-0.012529,-0.027775,0.014741,-0.111794,0.086426,-0.018620,-0.051513,-0.107239,0.008707,0.003339,-0.003314,0.277980,0.080273,-0.018096,0.058708,-0.006881,0.089443,0.019022,0.059850,0.001275,0.001224,0.057007,-0.080330,-0.006552,0.038003,-0.065069,-0.012807,-0.069361,0.115236,0.136525,-0.016363,-0.016032,-0.331328,0.073041,0.015697,-0.030485,-0.116836,0.003509,-0.020601,0.266587,-0.003554,-0.000452,-0.037212,-0.007643,-0.102960,0.100968,0.049805,-0.002057,0.006811,-0.037459,-0.048583,-0.080453,0.064363,0.037825,0.005799,0.084494,0.021466,0.071888,-0.032738,0.026694,-0.009055,-0.019654,-0.044413,-0.243743,0.273356,0.014407,-0.006452,0.002807,-0.072452,0.020635,0.012536,-0.002839,-0.010695,0.076789,-0.

In [ ]:
train.drop(train.columns[1:50], axis=1, inplace=True)

In [ ]:
train_df = train.melt(id_vars=['Page'], var_name='date')

In [ ]:
train_df.dropna(axis=0, how='any', inplace=True)

# construct ABT

In [ ]:
train_df = train_df.merge(page_df, how='left', on='Page')
train_df = train_df.merge(date_df.drop('date_str', axis = 1), how='left', on='date')

In [ ]:
melted_median = pd.read_pickle(root + 'melted_median_62.pkl')

train_df = train_df.merge(
    melted_median, 
    how='left', 
    on=['Page','date']
)

In [ ]:
# val_days = 62

In [ ]:
train_df['isval'] = (train_df.year == 2017) & (train_df.month >= 7)

In [ ]:
train_df.isval.sum()

In [ ]:
for c, dtype in zip(train_df.columns, train_df.dtypes):
    if dtype == np.float64:
        train_df[c] = train_df[c].astype(np.float32)
    if dtype == np.int64:
        train_df[c] = train_df[c].astype(np.int32)

In [ ]:
# del median_df, page_df, date_df, train
gc.collect()

# Preparing data and hyperparams

In [ ]:
name = 'gb7-r0'
num_searches = 1
boosting_rounds = 10000
stopping_rounds = 10
down_sample = None

In [ ]:
if down_sample is not None:
    train_df = train_df[train_df.index % down_sample == 0]

In [ ]:
for mname in median_name:
    train_df['median_{}'.format(mname)] = np.log1p(train_df['median_{}'.format(mname)])
for mname in median_name:
    if mname != '49_fix':
        train_df['median_diff_{}'.format(mname)] = train_df['median_{}'.format(mname)] - train_df['median_49_fix']
train_df['value'] = np.log1p(train_df['value']) - train_df['median_49_fix']

In [ ]:
train_df.head()

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    'metric': {'l1'},
    'num_leaves': 512,
#     'min_sum_hessian_in_leaf': 20,
    'max_depth': 12,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.85,
    'bagging_freq': 3,
    'verbose': 1,
#     'device' : 'gpu'
}

In [ ]:
train_df.sort_index(axis=1, inplace=True)

train = train_df[train_df.isval == False]
val = train_df[train_df.isval == True]

In [ ]:
drop_list = ['value', 'isval', 'Page', 'date']

lgb_train = lgb.Dataset(
    train.drop(drop_list, axis = 1), 
    train.value,
)
lgb_eval = lgb.Dataset(
    val.drop(drop_list, axis = 1), 
    val.value, 
    reference=lgb_train,
)

# del train_df, train
gc.collect()

In [ ]:
# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
# binary error
def SMAPE(preds, train_data):
    labels = train_data.get_label()
    y_true = np.round(np.expm1(labels))
    y_pred = np.round(np.expm1(preds))
    loss = np.mean(np.abs(y_true - y_pred) / np.maximum(1e-6, (np.abs(y_true) + np.abs(y_pred)))) * 200
    return 'SMAPE', loss, False

def SMAPE_2(preds, true):
    y_true = np.round(np.expm1(true))
    y_pred = np.round(np.expm1(preds))
    loss = np.mean(np.abs(y_true - y_pred) / np.maximum(1e-6, (np.abs(y_true) + np.abs(y_pred)))) * 200
    return loss

In [ ]:
results = []
for i in range(num_searches):
    print('Start LightGBM training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=boosting_rounds,
#                     feval=SMAPE,
                    valid_sets=[lgb_train, lgb_eval],
#                   categorical_feature=[],
                    early_stopping_rounds=stopping_rounds)

    print('Save model...')
    # save model to file
    gbm.save_model('model.{}.txt'.format(name))

    print('Plot feature importances...') 
    ax = lgb.plot_importance(gbm, max_num_features=100, importance_type='gain', title = 'gain', figsize=(20, 20))
    plt.show()
    ax = lgb.plot_importance(gbm, max_num_features=100, importance_type='split', title = 'split', figsize=(20, 20))
    plt.show()

In [ ]:
val_abt = val.drop(['value', 'isval', 'Page', 'date'], axis=1)
val_pred = gbm.predict(val_abt, num_iteration=gbm.best_iteration)
print('val SMAPE: ', SMAPE_2(val_pred + val.median_49_fix.values, val.value.values + val.median_49_fix.values))

- https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36780
- https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/38274#215155

# Prediction

In [ ]:
# name = 'gb4-r0'
# gbm = lgb.Booster(model_file='model.{}.txt'.format(name))

In [ ]:
test = pd.read_csv(root + 'key_2_modified.csv')

test_date_df = pd.read_csv(root + 'test_date_df.csv')
page_df = pd.read_pickle(root + 'page_ohe.pkl')

In [ ]:
median_data = []
median_name = [
    '49_fix', 'weekday_fix', 'weekend_fix', 
    'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5', 'dow6', 
    '28', 'weekday_28', 'weekend_28'
]
for mname in median_name:
    median_data.append(pd.read_pickle(root + 'median_{}.pkl'.format(mname)))
    
for i in range(len(median_data)):
    page_df['median_{}'.format(median_name[i])] = np.log1p(median_data[i].iloc[:, -1])
for i in range(len(median_data)):
    if i != 0:
        page_df['median_diff_{}'.format(median_name[i])] = page_df['median_{}'.format(median_name[i])] - page_df['median_49_fix']

In [ ]:
test = test.merge(page_df, how='left', on='Page')
test = test.merge(test_date_df, how='left', on='date')

for c, dtype in zip(test.columns, test.dtypes):
    if dtype == np.float64:
        test[c] = test[c].astype(np.float32)
    if dtype == np.int64:
        test[c] = test[c].astype(np.int32)

In [ ]:
test.isnull().sum().T

In [ ]:
test.sort_index(axis=1, inplace=True)

test_df = test.drop(['Page', 'date', 'Id'], axis=1)
pred = gbm.predict(test_df, num_iteration=gbm.best_iteration)

In [ ]:
visit = np.round(np.expm1(pred + test_df['median_49_fix'].values))
pred_df = pd.DataFrame({'Id':test.Id,'Visits':visit})

In [ ]:
pred_df.to_csv(
    os.path.join(root, 'test_prediction.{}.csv'.format(name)), index=False)